<a href="https://colab.research.google.com/github/Ttm2025/test/blob/main/KAz5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title /
from ast import Return
from IPython.display import display, Math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sympy as sp
import math
from functools import reduce
def add(x, y): return ['+', x, y]
def sub(x, y): return ['-', x, y]
def mul(x, y): return ['*', x, y]
def div(x, y): return ['/', x, y]
def power(x, y): return ['^', x, y]
def negative(x): return ['~', x]
def sin_(x): return ['sin', x]
def cos_(x): return ['cos', x]
def sqrt_(x): return ['sqrt', x]
def ln_(x): return ['ln', x]
def exp_(x): return ['exp', x]
def arcsin_(x): return ['arcsin', x]
def arccos_(x): return ['arccos', x]
def arctan_(x): return ['arctan', x]

def Sum(*x): return ['S', *x]
def Product(*x): return ['P', *x]

def fix(x):
  if isinstance(x, Expression):
    return x
  return Expression(x)

class Expression:
  def __init__(self, x):
    self.T = x
  def __add__(self, x):
    return Expression(add(self.T, fix(x).T))
  def __radd__(self, x):
    return Expression(add(fix(x).T, self.T))
  def __sub__(self, x):
    return Expression(sub(self.T, fix(x).T))
  def __rsub__(self, x):
    return Expression(sub(fix(x).T, self.T))
  def __mul__(self, x):
    return Expression(mul(self.T, fix(x).T))
  def __rmul__(self, x):
    return Expression(mul(fix(x).T, self.T))
  def __truediv__(self, x):
    return Expression(div(self.T, fix(x).T))
  def __rtruediv__(self, x):
    return Expression(div(fix(x).T, self.T))
  def __pow__(self, x):
    return Expression(power(self.T, fix(x).T))
  def __rpow__(self, x):
    return Expression(power(fix(x).T, self.T))
  def __neg__(self):
    return Expression(negative(self.T))  
def sin(x):
  return Expression(sin_(fix(x).T))
def cos(x):
  return Expression(cos_(fix(x).T))
def sqrt(x):
  return Expression(sqrt_(fix(x).T))
def ln(x):
  return Expression(ln_(fix(x).T))
def exp(x):
  return Expression(exp_(fix(x).T))
def arcsin(x):
  return Expression(arcsin_(fix(x).T))
def arccos(x):
  return Expression(arccos_(fix(x).T))
def arctan (x):
  return Expression(arctan_(fix(x).T))

def show(x):
  display(Math(latex(x)))

def enclose(x):
  return f'\\left( {x} \\right)'

def latex(x):
  if isinstance(x, Expression):
    return latex(x.T)

  if isinstance(x, int):
    return str(x)

  if isinstance(x, str):
    return extendSymbol(x)

  op = x[0]
  if op in '+-*/^':
    l, r = latex(x[1]), latex(x[2])
    if op == '+':
      return f'{l} + {r}'
    if op == '-':
      return f'{l} - {r}'
    if op == '*':
      if isinstance(x[1], list) and x[1][0] in '+-' or l[0] == '-':
        l = enclose(l)
      if isinstance(x[2], list) and x[2][0] in '+-' or r[0] == '-':
        r = enclose(r)      
      return f'{l} \\cdot {r}'
    if op == '/':
      return f'\\dfrac{{{l}}}{{{r}}}'
    if op == '^':
      #if r=='0':
       # return f'{{{1}}}'
      if r=='1':
        return f'{{{l}}}'
      #if l=='1':
       # return f'{{{1}}}'
      if isinstance(x[1], list):
        l = enclose(l)
      return f'{{{l}}}^{{{r}}}'
  elif op == 'S':
    A = [latex(a) for a in x[1:]]
    for k in range(len(A)):
      if A[k][0] == '-':
        A[k] = enclose(A[k])
      return ' + '.join(A)
  elif op == 'P':
    A = [latex(a) for a in x[1:]]
    for k in range(1, len(x)):
      if isinstance(x[k], list) and x[k][0] == 'S' or A[k-1][0] == '-':
        A[k-1] = enclose(A[k-1])
    return ' \\cdot '.join(A)
  elif op == 'q':
    if x[2] == 1:
      return str(x[1])
    if x[1] > 0:
      return f'\\dfrac{{{x[1]}}}{{{x[2]}}}'
    return f'-\\dfrac{{{-x[1]}}}{{{x[2]}}}'

  else: 
    a = enclose(latex(x[1]))
    if op == '~':
      return f'- {a}'
    if op == 'sin':
      return f'\\sin {a}'
    if op == 'cos':
      return f'\\cos {a}'
    if op == 'sqrt':
      return f'\\sqrt{{{a}}}'
    if op == 'ln':
      return f'\\ln {a}'
    if op == 'exp':
      return f'e ^ {{{a}}}'
    if op == 'arcsin':
      return f'\\arcsin {a}'
    if op == 'arccos':
      return f'\\arccos {a}'
    if op == 'arctan':
      return f'\\arctan {{{a}}}'

def extendSymbol(x):
  if len(x) == 1:
    return x
  if x in ['alpha', 'beta', 'gamma', 'pi']:
    return f'\\{x}'
  return f'{x[0]}_{{{x[1:]}}}'

def symbols(s):
  S = s.split()
  if len(S) == 1:
    return Expression(s)
  return (Expression(i) for i in S)

def evalf(x, arg = {}):
  if isinstance(x, Expression):
    if isinstance(arg, dict):
      arg = {i.T: arg[i] for i in arg}
    else:
      arg = {'$': arg}
    return evalf(x.T, arg)

  if isinstance(x, int):
    return x

  if isinstance(x, str):
    if x in arg:
      return arg[x]
    return arg['$']

  op = x[0]
  if op in '+-*/^':
    l, r = evalf(x[1], arg), evalf(x[2], arg)
    if op == '+':
      return l + r
    if op == '-':
      return l - r
    if op == '*':
      return l * r
    if op == '/':
      return l / r
    if op == '^':
      return l ** r
  elif op in 'SP':
    A = [evalf(a, arg) for a in x[1:]]
    if op == 'S':
      return sum(A)
    if op == 'P':
      return reduce(lambda x, y: x * y, A, 1)
  elif op == 'q':
    return x[1] / x[2]
  else:
    a = evalf(x[1], arg)
    if op == '~':
      return - a
    if op == 'sin':
      return np.sin(a)
    if op == 'cos':
      return np.cos(a)
    if op == 'sqrt':
      return np.sqrt(a)
    if op == 'ln':
      return np.log(a)
    if op == 'exp':
      return np.exp(a)
    if op == 'arcsin':
      return np.arcsin(a)
    if op == 'arccos':
      return np.arccos(a)
    if op == 'arctan':
      return np.arctan(a)
  return None

def Z(x):
  return Expression(x)

def isInt(x):
  return isinstance(x.T, int)

def diff(y, x):
  if isInt(y):
    return Z(0)

  if isinstance(y.T, str):
    if y.T == x.T:
      return Z(1)
    return Z(0)

  op = y.T[0] 
  if op in '+-*/^':
    u, v = Expression(y.T[1]), Expression(y.T[2])
    du, dv = diff(u, x), diff(v, x)
    if op == '+':
      return du + dv
    if op == '-':
      return du - dv
    if op == '*':
      #if isInt(u): return u * dv # упрощение
      #if isInt(v): return du * v # упрощение
      return du * v + u * dv
    if op == '/':
      if isInt(u): return - u * dv / v ** 2 # упрощение
      if isInt(v): return du / v # упрощение
      return (du * v - u * dv) / v ** 2
    if op == '^':
      if isInt(v): # u(x) ^ n
        n = v.T
        if n == 0:
          return Z(0)
        if n == 1:
          return du
        if du.T == 1: return n * u ** (n - 1) # упрощение
        return n * u ** (n - 1) * du
      if isInt(u): # n ^ v(x)
        n = u.T
        return ln(n) * (u ** v) * dv
  else:
    u = Expression(y.T[1])
    du = diff(u, x)
    if op == '~': return -du
    if op == 'sin': return cos(u) * du
    if op == 'cos': return -sin(u) * du
    if op == 'sqrt': return du / (2 * sqrt(u))
    if op == 'ln': return du / u
    if op == 'exp': return exp(u) * du
  
  return None

def Diff(y, x, n = 1):
  dy = y
  for i in range(n):
    dy = diff(dy, x)
  return dy

def taylorCoeffs(y, x, n, x0):
  A = [evalf(y, x0)]
  dy = diff(y, x)
  for i in range(n):
    A.append(evalf(dy, x0))
    dy = diff(dy, x)
  return A

def fac(n):
  return 1 if n < 2 else n * fac(n - 1) 

def taylorTerm(a, x, n, x0):
  a, x0 = int(a), int(x0)
  if n == 0:
    return Z(a)
  d = x
  if x0 > 0: 
    d = x - x0
  elif x0 < 0: 
    d = x + (-x0)
  if n == 1:
    return a * d
  return (Z(a) / fac(n)) * d ** n

def taylorSeries(y, x, n, x0 = 0):
  A = taylorCoeffs(y, x, n, x0)
  f = None
  for i in range(n + 1):
    if A[i] == 0:
      continue
    elif A[i] > 0:
      t = taylorTerm(A[i], x, i, x0)
      f = t if f == None else f + t
    else:
      t = taylorTerm(-A[i], x, i, x0)
      f = -t if f == None else f - t
  if f == None:
    f = Z(0)
  return f

def rational(n, m):
  if n == 0:
    return ['q', 0, 1]
  if m < 0:
    n, m = -n, -m
  d = gcd(abs(n), m)
  return ['q', n // d, m // d]

def Q(n, m):
  return Expression(rational(n, m))
  
def gcd(n, m):
  while m:
    n, m = m, n % m
  return n

def normalize(x):
  if isinstance(x, (int, str)): return x 
  op, arg = x[0], x[1:]
  A = [normalize(a) for a in arg] 
  if op in '+S': # сборка сумм
    R = []
    for a in A:
      R += a[1:] if isinstance(a, list) and a[0] in '+S' else [a]
    return Sum(*R)
  if op in '*P': # сборка произведений
    R = []
    for a in A:
      R += a[1:] if isinstance(a, list) and a[0] in '*P' else [a]
    return Product(*R)
  if op == '~': # унарный минус
    return normalize(Product(-1, A[0]))
  if op == '-': # бинарный минус
    return normalize(Sum(A[0], Product(-1, A[1])))
  if op == '/': # деление
    return normalize(Product(A[0], power(A[1], -1)))
  if op == '^': # нормализация степеней
    if isinstance(A[0], list) and A[0][0] == 'P':
      B = [power(a, A[1]) for a in A[0][1:]]
      return normalize(Product(*B))
    if isinstance(A[0], list) and A[0][0] == '^':
      d = Product(A[0][2], A[1])
      return normalize(power(A[0][1], d))
    #if A[-1] == 0:
    #  return rational(1, 1)
    #if A[-1]==1:
    #  return (A[0])
    #if A[0]==1:
    #  return 1
  return x

def sortx(x):
  if isinstance(x, list):
    op, arg = x[0], x[1:]
    A = [sortx(a) for a in arg]
    if op in 'SP':
      A.sort(key = lambda a: str(a))
    return [op, *A]
  return x

def simplify(x):
  if isinstance(x, Expression):
    if x.T[0]=='exp' and x.T[1]==0:
      return rational(1,1)
    if x.T[0]=='exp'and x.T[1][0]=='ln':
      f = x.T[1][1]
      return f
    if x.T[0]=='sin' and x.T[1]==0:
      return rational(0,1)
    if x.T[0]=='cos' and x.T[1]==0:
      return rational(1,1)
    if x.T[0]=='ln' and x.T[1]==1:
      return rational(1,1)
    if x.T[0]=='sin' and  x.T[1][0]=='arcsin' :
      f=x.T[1][1]
      if isinstance(f, int) and int(abs(f))>1:
           print("Искл: |x|>1")
      else:
         return f
    if x.T[0]=='arcsin' and  x.T[1][0]=='sin' :
      f=x.T[1][1]
      if isinstance(f, int) and int(abs(f))>math.pi/2:
           print("Искл: |x|>pi/2")
      else:
         return f
    if x.T[0]=='arccos' and  x.T[1][0]=='cos' :
      f=x.T[1][1]
      if isinstance(f, int) and (int((f))>math.pi or int((f))<0):
           print("Искл: x не принадлежит [0,pi] ")
      else:
         return f
    if x.T[0]=='cos' and  x.T[1][0]=='arccos' :
      f=x.T[1][1]
      if isinstance(f, int) and int(abs(f))>1:
           print("Искл: |x|>1")
      else:
         return f
    return Expression(simplify(x.T))
  if isinstance(x, str): # символ
    return x
  if isinstance(x, int): # целое, преобразуем в рациональное
      return rational(x, 1)
  if x[0] == 'q': # рациональное
    return x
  x = sortx(normalize(x))
  op, arg = x[0], x[1:]
  A = [simplify(a) for a in arg]
  if op == '^':
    if isinstance(A[0], list) and A[-1][1]==0:
      return rational(1,1)
    if isinstance(A[0], list) and A[0][1]==A[0][2]==1:
      return rational(1,1)
    if isinstance(A[1], list) and A[1][0] == 'q' and A[1][2] == 1:
      if isinstance(A[0], list) and A[0][0] == 'q':
        n, m, d = A[0][1], A[0][2], A[1][1]
        if d >= 0:
          return rational(n ** d, m ** d)
        else:
          return rational(m ** (-d), n ** (-d))
  if op == 'P':
    n, m, V = 1, 1, []
    for a in A:
      if isinstance(a, list) and a[0] == 'q':
        n, m = n * a[1], m * a[2]
      else:
        V.append(a)
    q = rational(n, m)
    if len(V) > 1:
      V = collectSameTermsP(V)
    # здесь можно упростить множители V, не являющиеся рациональными числами
    if q[1] == 0 or V == []: return q
    if q[1] == 1 and q[2] == 1 and len(V) == 1: return V[0]
    A = [q, *V] if q[1] != 1 or q[2] != 1 else [*V]
  if op == 'S':
    n, m, V = 0, 1, []
    for a in A:
      if isinstance(a, list) and a[0] == 'q':
        n, m = n * a[2] + m * a[1], m * a[2]
      else:
        V.append(a)
    q = rational(n, m)
    if len(V) > 1:
      V = collectSameTerms(V)
    if V == []: return q
    if q[1] == 0 and len(V) == 1: return V[0]
    A = [q, *V] if q[1] != 0 else [*V]
  return [op, *A]

def collectSameTerms(A):
  count, X = 1, []
  for k in range(1, len(A)):
    if A[k] == A[k - 1]:
      count += 1
      continue
    X.append(simplify(Product(rational(count, 1), A[k - 1])))
    count = 1
  X.append(simplify(Product(rational(count, 1), A[-1])))
  return X

def collectSameTermsP(A):
  count, X = 1, []
  for k in range(1, len(A)):
    if A[k] == A[k - 1]:
      count += 1
      continue
    X.append(simplify(Expression(A[k-1])**count))
  X.append(simplify(Expression(A[-1])**count))
  return X
      


In [ ]:
x=Expression('x')
y=Expression('y')
f = x * y + 2 + y * x 
show(f)
print(f.T)
F = Expression(normalize(f.T))
print(F.T)
F2 = Expression(sortx(F.T))
print(F2.T)
show(F2)
print()
f=x+x+x
F=Expression(normalize(f.T))
print(f.T)
show(simplify(F))
print()
f=x*x*x
F=Expression(normalize(f.T))
print(f.T)
show(simplify(F))
print()
f=(x+5)**0
F=Expression(normalize(f.T))
print(f.T)
show(simplify(F))
print()
f=(x)**1
F=Expression(normalize(f.T))
print(f.T)
show(simplify(F))
print()
f=(1)**(x+7)
F=Expression(normalize(f.T))
print(f.T)
show(simplify(F))
print()
f=exp(0)
F=Expression(normalize(f.T))
show(f)
show(simplify(F))
print()
f=sin(0)
F=Expression(normalize(f.T))
show(f)
show(simplify(F))
print()
f=cos(0)
F=Expression(normalize(f.T))
show(f)
show(simplify(F))
print()
f=ln(1)
F=Expression(normalize(f.T))
show(f)
show(simplify(F))
print()
f=exp(ln(1+x))
show(f)
F=Expression(normalize(f.T))
show(simplify(F))
print()
f=arcsin(sin(x))
show(f)
show(simplify(f))
print()
f=sin(arcsin(x))
show(f)
show(simplify(f))
print()
f=arccos(cos(x))
show(f)
show(simplify(f))
print()
f=cos(arccos(x+1))
show(f)
show(simplify(f))

print()
f=sin(arcsin(4))
show(f)
show(simplify(f))





<IPython.core.display.Math object>

['+', ['+', ['*', 'x', 'y'], 2], ['*', 'y', 'x']]
['S', ['P', 'x', 'y'], 2, ['P', 'y', 'x']]
['S', 2, ['P', 'x', 'y'], ['P', 'x', 'y']]


<IPython.core.display.Math object>


['+', ['+', 'x', 'x'], 'x']


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

Искл: |x|>1


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>